In [52]:
<<<<<<< LOCAL CELL DELETED >>>>>>>

housing_best_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 29) |>
  set_engine("kknn") |>
  set_mode("regression")

housing_fit <- workflow() |>
  add_recipe(housing_recipe) |>
  add_model(housing_best_spec) |>
  fit(data = training_full)

housing_summary <- housing_fit |>
  predict(testing_full) |>
  bind_cols(testing_full) |>
  metrics(truth = price, estimate = .pred) |>
  filter(.metric == 'rmse')
housing_summary

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
rmse,standard,711534.4
